# Persistent Cache

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

## Cache
Die Klasse `Cache` implementiert eine Transpositionstabelle die persistiert werden kann. Der Konstruktor erhält folgende Parameter: 
- `max_size` gibt an, wie viele Einträge sich maximal im Cache befinden dürfen, bevor `min_limit` erhöht wird und Einträge entfernt werden.
- `min_limit` gibt an, was das minimale Rekursionslimit sein muss, damit ein State im Cache gespeichert wird

In [ ]:
class Cache:
    def __init__(self, max_size = 1_000_000, min_limit = 0):
        self.min_limit = min_limit
        self.max_size = max_size
        self.cache = {}

Für Entwicklungszwecke wird eine Stringdarstellung für die Klasse `Cache` implementiert. Hierzu wird durch die Funktion `__repr__` ein String zurückgegeben, der alle Parameter der Klasse beinhaltet.

In [ ]:
def __repr__(self: Cache) -> str:
    return f"Cache(size={len(self.cache)}, max_size={self.max_size}, min_limit={self.min_limit})"

Cache.__repr__ = __repr__
del __repr__

Damit die Zustände und die Werte direkt als Bytes gespeichert und wieder ausgelesen werden können, ist das Paket `struct` nötig. Das Paket `tqdm` ermöglicht eine simple Fortschrittsanzeige.

In [ ]:
import struct
from tqdm.notebook import tqdm

Die Methode `convert_state_to_bytes` konvertiert einen Zustand in ein Byte-Array der Länge 7. Der Zustand wird dabei immer auf den weißen Spieler normiert. Dabei werden folgende Argumente erwartet:
- `state` $\in States$;
- `player` $\in Player$.

In [ ]:
def convert_state_to_bytes(state, player):
    players = ['w','b']
    if player == 'b':
        players = ['b','w']
    byte_data = ((state[0][0] << 4) | state[0][1]).to_bytes(1,'big')
    for player in players:
        for ring in range(2,-1,-1):
            ring_byte = 0
            for cell in range(7,-1,-1):
                ring_byte <<= 1
                if state[1][ring][cell] is player:
                    ring_byte |= 1
            byte_data += ring_byte.to_bytes(1,'big')
    return byte_data

Die Methode `write` schreibt einen Zustand in den Cache. Dabei wird der Zustand auf den weißen Spieler normiert. Es werden folgende Argumente erwartet:
- `state` $\in States$;
- `player` $\in Player$.
- `limit` $\in \mathbb{N}_0$;
- `value` $\in \mathopen[-1.0,1.0\mathclose]$;
- `alpha` $\in \mathopen[-1.0,1.0\mathclose]$;
- `beta` $\in \mathopen[-1.0,1.0\mathclose]$;

In [ ]:
def write(self, state, player: str, limit: int, value: float, alpha: float, beta: float) -> None:
    state = convert_state_to_bytes(state, player)
    key = state + limit.to_bytes(1,'big')
    if player == 'b':
        value, alpha, beta = -value, -beta, -alpha
    value = struct.pack("d", value) + struct.pack("d",alpha) + struct.pack("d",beta)
    self.cache[key] = value

Cache.write = write
del write

Die Methode `read` liest einen vorher gespeicherten Zustand aus dem Cache aus. Falls der Zustand nicht vorhanden ist wird `None` zurück gegeben. Dabei wird die Normierung auf den weißen Spieler rückgängig gemacht. Folgende Argumente werden erwartet:
- `state` $\in States$;
- `player` $\in Player$.
- `limit` $\in \mathbb{N}_0$;

Zurückgegeben wird ein Tripel bestehend aus:
1. `value` $\in \mathopen[-1.0,1.0\mathclose]$;
2. `alpha` $\in \mathopen[-1.0,1.0\mathclose]$;
3. `beta` $\in \mathopen[-1.0,1.0\mathclose]$;

In [ ]:
def read(self, state, player: str, limit: int) -> (float,float,float):
    state = convert_state_to_bytes(state, player)
    key = state + limit.to_bytes(1,'big')
    result = self.cache.get(key)
    if not result:
        return None
    value = struct.unpack("d", result[:8])[0]
    alpha = struct.unpack("d", result[8:16])[0]
    beta = struct.unpack("d", result[16:24])[0]
    if player == 'b':
        value, alpha, beta = -value, -beta, -alpha
    return (value, alpha, beta)

Cache.read = read
del read

Die Methode `clean` prüft ob der Cache seine maximale Größe überschritten hat. Ist dies der Fall wird das minimale Rekursionslimit um eins erhöht und alle Einträge, deren Rekursionslimit geringer ist, werden aus dem Cache entfernt.

In [ ]:
def clean(self):
    cleaned = False
    while len(self.cache) > self.max_size:
        self.min_limit += 1
        pre_len = len(self.cache)
        self.cache = {
            key: value
            for key, value in self.cache.items()
            if key[7] >= self.min_limit
        }
        cleaned = True
        print(f"Increased min_limit to {self.min_limit} and deleted {pre_len - len(self.cache)} entries. Cache is now {len(self.cache)} entries big.")
    return cleaned
Cache.clean = clean
del clean

Die Methode `save` persistiert den Cache auf dem Dateisystem des Computers. Dafür wird folgendes Argument erwartet:
- `path` beschreibt den Pfad zur Cache-Datei im Dateisystem;

In [ ]:
def save(self, path: str):
    with open(path, "wb") as file:
        for key,value in tqdm(self.cache.items()):
            file.write(key)
            file.write(value)
Cache.save = save
del save

Die Methode `load` lädt einen vorhandenen Cache basierend auf einer Cache-Datei, die sich auf dem Dateisystem des Computers befindet. Dafür wird folgendes Argument erwartet:
- `path` beschreibt den Pfad zur Cache-Datei im Dateisystem;

In [ ]:
def load(self, path: str):
    if not os.path.isfile(path):
        return
    with open(path, "rb") as file:
        while True:
            key = file.read(9)
            value = file.read(24)
            if not key or not value:
                break
            self.cache[key] = value

Cache.load = load
del load